In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop('User ID',axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [8]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [9]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [10]:
#From above count we can see this is imbalanced data

In [11]:
indep = dataset[['Age', 'EstimatedSalary',  'Gender_Male']]

In [12]:
dep = dataset[['Purchased']]

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
xtrain,xtest,ytrain,ytest = train_test_split(indep,dep,test_size=1/3,random_state=0)

In [15]:
from sklearn.linear_model import LogisticRegression

In [34]:
#param = {'solver':['lbfgs','liblinear','newton-cg','newton-cholesky','sag','saga'],'multi_class':['auto','ovr','multinomial'],
 #        'penalty':['l1','l2','elasticnet', None]}
param = {'solver':['lbfgs','liblinear','newton-cg','newton-cholesky','sag','saga'],
         'penalty':['l2']}

In [35]:
from sklearn.model_selection import GridSearchCV


In [61]:
#model= LogisticRegression(random_state=0)
model = GridSearchCV(LogisticRegression(),param,refit=True,verbose=3,scoring='f1_weighted',n_jobs=-1)

In [37]:
model.fit(xtrain,ytrain.values.ravel())

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             scoring='f1_weighted', verbose=3)

In [38]:
ypredict = model.predict(xtest)

In [57]:
model.best_params_

{'penalty': 'l2', 'solver': 'lbfgs'}

In [40]:
re = model.cv_results_

In [46]:
table = pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.048211,0.036397,0.019346,0.018156,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
1,0.004340,0.001364,0.006226,0.002027,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.509779,0.525300,0.50141,0.50141,0.501410,0.507862,0.009302,4
2,0.134454,0.037300,0.004765,0.002490,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
3,0.008937,0.001329,0.004813,0.000725,l2,newton-cholesky,"{'penalty': 'l2', 'solver': 'newton-cholesky'}",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
4,0.010357,0.003088,0.007455,0.001928,l2,sag,"{'penalty': 'l2', 'solver': 'sag'}",0.509779,0.525300,0.50141,0.50141,0.501410,0.507862,0.009302,4
5,0.009806,0.001119,0.004882,0.001243,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.509779,0.525300,0.50141,0.50141,0.501410,0.507862,0.009302,4


In [47]:
from sklearn.metrics import confusion_matrix

In [48]:
clf = confusion_matrix(ytest,ypredict)

In [49]:
print(clf)

[[79  6]
 [ 9 40]]


In [50]:
from sklearn.metrics import classification_report

In [51]:
clf_rep = classification_report(ytest,ypredict)

In [52]:
print(clf_rep)

              precision    recall  f1-score   support

           0       0.90      0.93      0.91        85
           1       0.87      0.82      0.84        49

    accuracy                           0.89       134
   macro avg       0.88      0.87      0.88       134
weighted avg       0.89      0.89      0.89       134



In [54]:
from sklearn.metrics import f1_score
f1 = f1_score(ytest,ypredict,average='weighted')

In [55]:
print (f1)

0.8872628037179481


In [58]:
from sklearn.metrics import roc_auc_score
f1 = roc_auc_score(ytest,model.predict_proba(xtest)[:,1])

In [59]:
f1

0.9510204081632653